In [ ]:
# 화면 가로 확장 코드 (기본 width 50%)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

### 데이터 판다스로 입력, 컬럼 정리

In [ ]:
import pandas as pd
ch1 = pd.read_csv('../../../channel_1.dat', names=['Unix', 'Watt'],header=None, delimiter=" ")
print('Data File Import Done.')

In [ ]:
ch1.info()

In [ ]:
from datetime import datetime
ch1_dated = pd.DataFrame(ch1,columns = ['Unix', 'Date', 'Watt'])

ch1_dated['Date'] = pd.to_datetime(ch1['Unix'], unit='s')
ch1_dated.info()
ch1_dated.head(10)

### 변수 및 구분 단계

### csv 파일 저장을 위한 폴더 생성

In [ ]:
import os
try:
    os.mkdir('./New_Electirc_Data_CSV')
except FileExistsError:
    print('해당 폴더가 이미 존재합니다.\n폴더를 확인해주세요.')

In [ ]:
"""
while(전체)
    while(year)
        while(month)
            while(day) -> 각 월의 일수만큼 실행 필요,, 일 비교후 결손치를 0 1440개 삽입 (미완)
                while(hour) -> 24개 실행 필요, 시 비교 후 결손치 0 60개 삽입 (미완)
                    while(minute) -> 60개 실행 필요,, 분 비교후 결손치 0 삽입 (완)
                        while(True) -> 초단위, 데이터가 존재한다면 무조건 실행
"""

### 실험안

In [ ]:
import csv
import time
import sys
month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

TARGET_YEAR_START = 2013
TARGET_YEAR_END=2013

def value (data, num):
    list0 = data.loc[num].values.tolist()
    date = list0[1]
    YYYY = date.year
    MM = date.month
    DD = date.day
    hh = date.hour
    mm = date.minute
    watt = list0[2]
    return YYYY, MM, DD, hh, mm, watt, date

print(value(ch1_dated,431369)[0])

#i1=0
i1=431369
while(i1 < len(ch1_dated)):
    print("전체 과정 시작")
    YEAR0 = value(ch1_dated, i1)[0]

    if(YEAR0>=TARGET_YEAR_START and YEAR0<=TARGET_YEAR_END):
        print("년 단위 시작")
        MONTH0 = value(ch1_dated, i1)[1]
        
        while(MONTH0<13):
            print("월 단위 시작")
            MONTH1 = value(ch1_dated, i1)[1]
            DAY0 = value(ch1_dated, i1)[2]
            
            REAL_DAY = 1
            ONE_MONTH_LIST=[]
            while(REAL_DAY<month[MONTH1]):
                print("<<<{0}일 데이터 시작 {1}>>>".format(REAL_DAY, value(ch1_dated, i1)[6]),i1)
                MONTH2 = value(ch1_dated, i1)[1]
                DAY1 = value(ch1_dated, i1)[2]
                HOUR0 = value(ch1_dated, i1)[3]
                
                REAL_HOUR = 0
                ONE_DAY_LIST=[]
                while(REAL_HOUR<24):
                    print("<<{0}시 데이터 시작 {1}>>".format(REAL_HOUR, value(ch1_dated, i1)[6]),i1)
                    DAY2 = value(ch1_dated, i1)[2]
                    HOUR1 = value(ch1_dated, i1)[3]

                    REAL_MINUTE = 0
                    MIN_60_LIST=[]
                    while(REAL_MINUTE<60):#특정 시간대 안에서 분단위 데이터를 60개 처리
                        DAY3 = value(ch1_dated, i1)[2]
                        HOUR2 = value(ch1_dated, i1)[3]
                        MINUTE1 = value(ch1_dated, i1)[4]

                        print("<{0}분 데이터 시작 {1}>".format(REAL_MINUTE, value(ch1_dated, i1)[6]),i1)
                        SEC_60_LIST=[]

                        if MINUTE1==REAL_MINUTE:
                            while(True):#특정 분대 안에서 초단위 데이터를 처리
                                DAY4 = value(ch1_dated, i1)[2]
                                HOUR3 = value(ch1_dated, i1)[3]
                                MINUTE2 = value(ch1_dated, i1)[4]
                                SEC_60_LIST.append(value(ch1_dated, i1)[5])
                                i1+=1

                                if MINUTE1!=MINUTE2:
                                    if MINUTE2-MINUTE1==1 or MINUTE1-MINUTE2==59: #직후 분으로 넘어가거나 59분에서 다음 시로 넘어가는 경우
                                        print("*{0}분의 값들 >> {1}\n".format(MINUTE1, SEC_60_LIST))
                                        i1-=1
                                        break

                                    else:#진행되다가 분대가 넘어가긴 했으나 다음 분으로 넘어가지 않고 점프해버린경우
                                        print("*{0}분의 값들 >> {1} (결손 발생)\n".format(MINUTE1, SEC_60_LIST))
                                        break

                        if MINUTE1!=REAL_MINUTE:#아예 해당분대가 결손된 경우
                            SEC_60_LIST.append(0)
                            print("{0}분의 결손치는 0으로 처리됨.\n".format(REAL_MINUTE))

                        MIN_60_LIST.append(max(SEC_60_LIST))
                        REAL_MINUTE+=1

                    if REAL_MINUTE==60 and len(MIN_60_LIST)==60:
                        pass
                        for i in MIN_60_LIST:
                                ONE_DAY_LIST.append(i)
                        print("*{0}시의 분별 최댓값 {1}개 >> {2}\n".format(REAL_HOUR, len(MIN_60_LIST), MIN_60_LIST))

                    if REAL_MINUTE==60 and len(MIN_60_LIST)<60:
                        for i in range(60-len(MIN_60_LIST)):
                            MIN_60_LIST.append(0)
                        for i in MIN_60_LIST:
                                ONE_DAY_LIST.append(i)
                        print("*{0}시의 분별 최댓값 {1}개 >> {2}\n".format(REAL_HOUR, len(MIN_60_LIST), MIN_60_LIST))

                    REAL_HOUR+=1
                
                #print("<여기는 하루단위>", REAL_HOUR, len(ONE_DAY_LIST))

                if REAL_HOUR==24 and len(ONE_DAY_LIST)==1440:
                    for i in ONE_DAY_LIST:
                            ONE_MONTH_LIST.append(i)
                    print("*{0}일의 분별 최댓값 {1}개 추가됨.\n".format(REAL_DAY, len(ONE_DAY_LIST)))

                if REAL_HOUR==24 and len(ONE_DAY_LIST)<1440:
                    for i in range(1440-len(ONE_DAY_LIST)):
                        ONE_DAY_LIST.append(0)
                    for i in ONE_DAY_LIST:
                            ONE_MONTH_LIST.append(i)
                    print("*{0}일의 분별 최댓값 {1}개 추가됨.\n".format(REAL_DAY, len(ONE_DAY_LIST)))

                REAL_DAY+=1
                """
                
                if REAL_DAY==month[MONTH1]

                            if REAL_HOUR!=HOUR3 and REAL_HOUR==23:#정상 처리 경우, 하루 마무리하고 다음 일자로 넘어간 경우
                                print("*{0}시의 분별 최댓값 {1}개 >> {2}\n".format(HOUR2, len(MIN_60_LIST), MIN_60_LIST))
                                for i in MIN_60_LIST:
                                    ONE_DAY_LIST.append(i)
                                #여기서는 하루의 1분 단위 데이터가 전부 모인 상태이니 5분 단위 Max값 추출이나 10분 단위 추출을 진행해서
                                #한달의 데이터 리스트에 담길 하루의 데이터 리스트로 편집할것! => 5냐 10이냐에 따른 if문이 들어갈 계획.
                                print("*{0}일의 하루 데이터 {1}개 리스트\n>>{2}\n".format(REAL_DAY, len(ONE_DAY_LIST), ONE_DAY_LIST))
                                break
                        

                        if HOUR1!=REAL_HOUR and HOUR1-REAL_HOUR!=1:
                            for i in range(HOUR1-REAL_HOUR-1):
                                for i in range(60):
                                    ONE_DAY_LIST.append(0)
                                REAL_HOUR+=1
                                print("*{0}시의 분별 최댓값 60개 >> 0으로 처리\n".format(REAL_HOUR))
                        
                    
                        
                        #여기에 달이 바뀌는 경우 코드 들어가야 할 듯 한데....일단 1월말이랑 3월까지 해서 처리해보자
                """
                
    else:
        i1+=1
        pass

### 보관안

In [ ]:
import csv
import time
import sys
month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

"""
print("해당 전력 데이터는 2012년 11월 10일부터 2017년 4월 25일까지 온전한 데이터가 존재합니다.\n")
time.sleep(0.5)
print("데이터는 사용자가 선택한 기간 중 1년 단위로 처리됩니다.\n")
time.sleep(0.5)
global TARGET_YEAR_START 
TARGET_YEAR_START = int(input("목표로 하는 처리 기간의 시작 연도를 4자리 숫자로 입력해주세요. : \n"))
time.sleep(0.5)
global TARGET_YEAR_END
TARGET_YEAR_END = int(input("목표로 하는 처리 기간의 종료 연도를 4자리 숫자로 입력해주세요. : \n"))
time.sleep(0.5)

if(TARGET_YEAR_START<2012 or TARGET_YEAR_START>2017 or TARGET_YEAR_END>2017 or  TARGET_YEAR_END<2012):
    print("\---기간을 잘못 설정했습니다!---")
    print("---프로그램을 종료합니다.---")
    sys.exit()

if(TARGET_YEAR_START==2012 or TARGET_YEAR_END==2017):
    print("\n---해당 기간은 현재 서비스하지 않습니다!---")
    print("---프로그램을 종료합니다.---")
    sys.exit()

global HANDLE_MINUTE
HANDLE_MINUTE = int(input("데이터들은 기본적으로 1분 단위 최댓값으로 정리됩니다.\n추후 가공될 분 단위를 5분과 10분 중 골라 숫자만 입력해주세요. : \n"))
if(HANDLE_MINUTE!=5 and HANDLE_MINUTE!=10):
    print("\n---잘못된 시간 처리 단위를 선택했습니다!---")
    print("---프로그램을 종료합니다.---")
    sys.exit()
"""

TARGET_YEAR_START = 2013
TARGET_YEAR_END=2013

def value (data, num):
    list0 = data.loc[num].values.tolist()
    date = list0[1]
    YYYY = date.year
    MM = date.month
    DD = date.day
    hh = date.hour
    mm = date.minute
    watt = list0[2]
    return YYYY, MM, DD, hh, mm, watt, date

print(value(ch1_dated,431369)[0])

#i1=0
i1=431369
while(i1 < len(ch1_dated)):
    print("전체 과정 시작")
    YEAR0 = value(ch1_dated, i1)[0]

    if(YEAR0>=TARGET_YEAR_START and YEAR0<=TARGET_YEAR_END):
        print("년 단위 시작")
        MONTH0 = value(ch1_dated, i1)[1]
        
        while(MONTH0<13):
            print("월 단위 시작")
            MONTH1 = value(ch1_dated, i1)[1]
            DAY0 = value(ch1_dated, i1)[2]
            
            REAL_DAY = 0
            while(REAL_DAY<month[MONTH1]):
                REAL_DAY+=1
                print("{0}일 데이터 시작 {1}".format(REAL_DAY, value(ch1_dated, i1)[6]),i1)
                MONTH2 = value(ch1_dated, i1)[1]
                DAY1 = value(ch1_dated, i1)[2]
                HOUR0 = value(ch1_dated, i1)[3]
                
                if DAY1==REAL_DAY:
                    REAL_HOUR = 0
                    ONE_DAY_LIST=[]
                    while(REAL_HOUR<24):
                        print("{0}시 데이터 시작 {1}".format(REAL_HOUR, value(ch1_dated, i1)[6]),i1)
                        DAY2 = value(ch1_dated, i1)[2]
                        HOUR1 = value(ch1_dated, i1)[3]
                        #MINUTE0 = value(ch1_dated, i1)[4] 삭제 고려

                        REAL_MINUTE = 0
                        MIN_60_LIST=[]
                        while(REAL_MINUTE<60):#특정 시간대 안에서 분단위 데이터를 60개 처리
                            DAY3 = value(ch1_dated, i1)[2]
                            HOUR2 = value(ch1_dated, i1)[3]
                            MINUTE1 = value(ch1_dated, i1)[4]

                            print("{0}분 데이터 시작 {1}".format(REAL_MINUTE, value(ch1_dated, i1)[6]),i1)
                            SEC_60_LIST=[]

                            if MINUTE1==REAL_MINUTE:
                                while(True):#특정 분대 안에서 초단위 데이터를 처리
                                    DAY4 = value(ch1_dated, i1)[2]
                                    HOUR3 = value(ch1_dated, i1)[3]
                                    MINUTE2 = value(ch1_dated, i1)[4]

                                    SEC_60_LIST.append(value(ch1_dated, i1)[5])
                                    i1+=1
                                    print("MINUTE1>>{0}, MINUTE2>>{1}, DAY4>>{2}, REAL_DAY>>{3}".format(MINUTE1, MINUTE2, DAY4, REAL_DAY))

                                    if MINUTE1!=MINUTE2:
                                        if MINUTE2-MINUTE1==1 or MINUTE1-MINUTE2==59: #직후 분으로 넘어가거나 59분에서 다음 시로 넘어가는 경우
                                            print("*{0}분의 값들 >> {1}\n".format(MINUTE1, SEC_60_LIST))
                                            i1-=1
                                            break
                                            
                                        else:#진행되다가 분대가 넘어가긴 했으나 다음 분으로 넘어가지 않고 점프해버린경우
                                            print("*{0}분의 값들 >> {1} (결손 발생)\n".format(MINUTE1, SEC_60_LIST))
                                            break

                            if MINUTE1!=REAL_MINUTE:#아예 해당분대가 결손된 경우
                                SEC_60_LIST.append(0)
                                print("{0}분의 결손치는 0으로 처리됨.\n".format(REAL_MINUTE))

                            MIN_60_LIST.append(max(SEC_60_LIST))

                            if REAL_HOUR!=HOUR3 and DAY4==REAL_DAY:#같은 일자 안에서 다음 시간대로 넘어간 경우
                                if len(MIN_60_LIST)==60:#60개 분에 대한 데이터가 모두 있으면 그냥 Pass
                                    pass
                                if len(MIN_60_LIST)<60:#분별 데이터가 60개 미만인 경우 나머지는 0으로 채움. (중간에 빈 경우는 위에서 따로 처리했음.)
                                    for i in range(60-len(MIN_60_LIST)):
                                        MIN_60_LIST.append(0)
                                for i in MIN_60_LIST:
                                        ONE_DAY_LIST.append(i)
                                print("*{0}시의 분별 최댓값 {1}개 >> {2}\n".format(HOUR2, len(MIN_60_LIST), MIN_60_LIST))
                                break

                            if REAL_HOUR!=HOUR3 and REAL_HOUR==23:#정상 처리 경우, 하루 마무리하고 다음 일자로 넘어간 경우
                                print("*{0}시의 분별 최댓값 {1}개 >> {2}\n".format(HOUR2, len(MIN_60_LIST), MIN_60_LIST))
                                for i in MIN_60_LIST:
                                    ONE_DAY_LIST.append(i)
                                #여기서는 하루의 1분 단위 데이터가 전부 모인 상태이니 5분 단위 Max값 추출이나 10분 단위 추출을 진행해서
                                #한달의 데이터 리스트에 담길 하루의 데이터 리스트로 편집할것! => 5냐 10이냐에 따른 if문이 들어갈 계획.
                                print("*{0}일의 하루 데이터 {1}개 리스트\n>>{2}\n".format(REAL_DAY, len(ONE_DAY_LIST), ONE_DAY_LIST))
                                break
                            REAL_MINUTE+=1

                        if HOUR1!=REAL_HOUR and HOUR1-REAL_HOUR!=1:
                            for i in range(HOUR1-REAL_HOUR-1):
                                for i in range(60):
                                    ONE_DAY_LIST.append(0)
                                REAL_HOUR+=1
                                print("*{0}시의 분별 최댓값 60개 >> 0으로 처리\n".format(REAL_HOUR))
                        
                        REAL_HOUR+=1
                        
                        #여기에 달이 바뀌는 경우 코드 들어가야 할 듯 한데....일단 1월말이랑 3월까지 해서 처리해보자
                """
                if DAY1!=REAL_DAY and REAL_DAY-DAY1!=1:
                    if MONTH1==MONTH2:
                        print("DAY0 >>{0}, DAY1>>{1}, DAY2>>{2}, REAL_DAY>>{3}".format(DAY0, DAY1, DAY2, REAL_DAY))
                        print("1번 경우")
                        sys.exit()
                    if MONTH1!=MONTH2:
                        print("2번 경우")
                        sys.exit()"""
                
                
    else:
        i1+=1
        pass

### 아래가 원본

In [ ]:
import csv
import time

month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

# 1. 
i = 431369#431369가 맞음
print('Running!', '\n'*4)


# 2. 
while(i < 20284837):
    try:
        list1 = ch1_date.loc[i].values.tolist()
        date = list1[1]
        y1 = date.year
        M1 = date.month
        d1 = date.day
        H1 = date.hour 
        n = 0 

        # 3. 
        while(H1 < 24):
            H0 = 0 

            # 3'. 
            while(H0 < 24):
                watt_list=[]
                h = date.hour

                # 3.1. 
                if(H1 == H0):

                    # 3.1'. 
                    while(h < h+1):
                        n=n+1
                        list2 = ch1_date.loc[i].values.tolist()
                        date = list2[1]
                        watt = list2[2]
                        y = date.year
                        M = date.month
                        d = date.day
                        h = date.hour
                        m = date.minute
                        s = date.second

                        j=h-H1

                        # 3.1'.1. 
                        if(j==0):
                            watt_list.append(watt)
                            i = i+1

                        # 3.1'.2.
                        elif (j==1):
                            max_watt = max(watt_list)

                            hourly_date2 = '{0}년 {1}월 {2}일 - {3}시'.format(y, M, d, H1)
                            hourly_date5 = '{0} {1} {2}'.format(y, M, d) 
                            hourly_list = []
                            hourly_list.append(hourly_date2)
                            hourly_list.append(max_watt)

                            with open("./Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(hourly_list)       

                            H1 = H1+1 
                            H0 = H0+1
                            break

                        # 3.1'.3. 
                        elif(d1+1==d and h ==0 and H1 ==23):
                            d=d-1
                            h=h+24

                            max_watt = max(watt_list)

                            hourly_date3 = '{0}년 {1}월 {2}일 - {3}시'.format(y, M, d, H1)
                            hourly_list = []
                            hourly_list.append(hourly_date3)
                            hourly_list.append(max_watt)

                            with open("./Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(hourly_list)       

                            H1 = H1+1 
                            H0 = H0+1                   
                            break

                        # 3.1'.4.
                        elif (n>600):
                            list(hourly_date5)
                            listed = hourly_date5.split()
                            y4 = listed[0]
                            M4 = listed[1]
                            d4 = listed[2]

                            max_watt = max(watt_list)

                            hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, d4, H1)
                            hourly_list = []
                            hourly_list.append(hourly_date)
                            hourly_list.append(max_watt)

                            with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(hourly_list)       

                            # 3.1'.4.1. 
                            if(d1==d):
                                jc=j

                                while(jc>1):
                                    h=h+1
                                    hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, d4, h-j)
                                    hourly_list = []
                                    hourly_list.append(hourly_date)
                                    hourly_list.append('Null')

                                    with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(hourly_list)  

                                    jc=jc-1
                                    H1 = H1+1
                                    H0 = H0+1

                                H1 = H1+1
                                H0 = H0+1

                            # 3.1'.4.2.
                            if(d1!=d):

                                while(H1<23):
                                    hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, d4, H1+1)
                                    hourly_list = []
                                    hourly_list.append(hourly_date)
                                    hourly_list.append('Null')

                                    with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(hourly_list)  

                                    H1 = H1+1
                                    H0 = H0+1

                                H1 = H1+1
                                H0 = H0+1

                                # 3.1'.4.2.1. 
                                if(int(d4)+1 != d):
                                    D = int(d4)+1 

                                    # 3.1'.4.2.1.1. 
                                    if(int(M) == int(M4)):
                                        while(D<d):
                                            for x in range (0,24):
                                                hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, D, x)
                                                hourly_list = []
                                                hourly_list.append(hourly_date)
                                                hourly_list.append('Null')

                                                with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                                    writer = csv.writer(file)
                                                    writer.writerow(hourly_list)  

                                            D=D+1

                                    # 3.1'.4.2.1.2.
                                    if(int(M) != int(M4)):
                                        while(D<=month[int(M4)]):
                                            for x in range (0,24):
                                                hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, D, x)
                                                hourly_list = []
                                                hourly_list.append(hourly_date)
                                                hourly_list.append('Null')

                                                with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                                    writer = csv.writer(file)
                                                    writer.writerow(hourly_list)  

                                            D=D+1

                                        D2 = 1
                                        while(D2 < d):
                                            for x in range (0,24):
                                                hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M, D2, x)
                                                hourly_list = []
                                                hourly_list.append(hourly_date)
                                                hourly_list.append('Null')

                                                with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                                    writer = csv.writer(file)
                                                    writer.writerow(hourly_list)  

                                            D2 = D2+1

                            break


                # 3.2. 
                if(H1 != H0):
                    list3 = ch1_date.loc[i].values.tolist()
                    date = list3[1]
                    watt = list3[2]
                    y = date.year
                    M = date.month
                    d = date.day
                    h = date.hour
                    m = date.minute
                    s = date.second

                    j=h-H1           

                    hourly_date3121 = '{0}년 {1}월 {2}일 - {3}시'.format(y, M, d, H0)
                    hourly_list3121 = []
                    hourly_list3121.append(hourly_date3121)
                    hourly_list3121.append('Null')

                    with open("./Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(hourly_list3121)       

                    h = h+1
                    H0 = H0+1

                    if (H0 ==23 and h!=24 and H1!=23):
                                break

                    if(h==24 and H0==23 and H1==23):
                        hourly_date3121 = '{0}년 {1}월 {2}일 - 23시'.format(y, M, d)
                        hourly_list3121 = []
                        hourly_list3121.append(hourly_date3121)
                        hourly_list3121.append('Null')

                        with open("./Ch"+str(Total_num)+"_Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch"+str(Total_num)+"_Electric_Data.csv", 'a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(hourly_list3121)

            break
    except:
        pass
    
print('Job Done!!')
time.sleep(5)